In [1]:
from bs4 import BeautifulSoup
import requests

In [2]:
html_doc = """
<html><head><title>Web scraping pro datovou analýzu v Pythonu - sedlakovi.org</title></head>
<body>
<p class="title"><b>Web scraping pro datovou analýzu v Pythonu</b></p>

<p class="description">
    Náš kurz Vás naučí jak lze snadno získávat data z webových stránek, 
    které můžete používat k následné analýze anebo reportingu. 
    Příkladem může být analýza kurzu měn, recenzí kaváren nebo cen zboží. 
    Náš kurz Vám ukáže celý postup od identifikace datového zdroje, 
    navržení vhodného scrapingu a přípravy dat pro následnou analýzu. 
</p>

<p class="description">
    Naučíte se:
    <ul>
        <li>Strukturu HTML</li>
        <li>Jak získat obsah webových stránek (knihovna requests)</li>
        <li>Jak vybrat z tohoto obsahu jenom to, co nás zajímá (knihovna BeautifulSoup)</li>
        <li>Jak následně data zpracovat (knihovna pandas)</li>
    </ul>
</p>

<table>
<tr> <th class="table-header"> Naučím se pracovat s: </th>
<td class="table-data">
<a href="http://docs.python-requests.org/en/master/" class="library" id="link1">Requests</a>,
<a href="https://www.crummy.com/software/BeautifulSoup/"  class="library" id="link2">BeautifulSoup</a>,
<a href="https://pandas.pydata.org/" class="library" id="link3">Pandas</a>
</td>
</tr>
</table>
"""

In [3]:
soup = BeautifulSoup(html_doc, 'html.parser')

# Cvičení 3.1

Vyberte odstavec s třídou `title`.

`Web scraping pro datovou analýzu v Pythonu`

In [4]:
soup.find(class_='title').text

'Web scraping pro datovou analýzu v Pythonu'

In [5]:
soup.find(attrs={'class': 'title'}).text

'Web scraping pro datovou analýzu v Pythonu'

In [6]:
soup.find(name='p', attrs={'class': 'title'}).text

'Web scraping pro datovou analýzu v Pythonu'

# Cvičení 3.2

Vyberte všechny položky seznamu

```
 <li>Strukturu HTML</li>
 <li>Jak získat obsah webových stránek (knihovna requests)</li>
 <li>Jak vybrat z tohoto obsahu jenom to, co nás zajímá (knihovna BeautifulSoup)</li>
 <li>Jak následně data zpracovat (knihovna pandas)</li>
```

** Bonus:** zbavte se HTML tagů (např. `Jak získat obsah webových stránek (knihovna requests)`). 

In [7]:
soup.find_all('li')

[<li>Strukturu HTML</li>,
 <li>Jak získat obsah webových stránek (knihovna requests)</li>,
 <li>Jak vybrat z tohoto obsahu jenom to, co nás zajímá (knihovna BeautifulSoup)</li>,
 <li>Jak následně data zpracovat (knihovna pandas)</li>]

In [8]:
[li.text for li in soup.find_all('li')]

['Strukturu HTML',
 'Jak získat obsah webových stránek (knihovna requests)',
 'Jak vybrat z tohoto obsahu jenom to, co nás zajímá (knihovna BeautifulSoup)',
 'Jak následně data zpracovat (knihovna pandas)']

# Cvičení 5.1

Zjistěte na stránkach `https://www.seznam.cz/`, kdo má dnes svátek.

In [9]:
url = 'https://www.seznam.cz/'
r = requests.get(url)
soup = BeautifulSoup(r.text, 'html.parser')
soup.find(class_='name-link').text

'Jáchym'

# Cvičení 5.2

Najděte které filmy se promítaly 8.8.2019 v kině Aero (první v seznamu abecedně seřazených kinech).

`Je mi fajn s.r.o.`

In [10]:
url = 'https://dokina.tiscali.cz/program-kin?date=2019-08-08&cinemasType=nonmulti&tab=cinemas&cinemasOrdering=title'
r = requests.get(url)
soup = BeautifulSoup(r.text, 'html.parser')

In [11]:
program = soup.select_one('.program')

In [12]:
program.select_one('.cinema-info .name').text

'Aero'

In [13]:
program.select_one('.program-table .name').text

'Je mi fajn s.r.o.'

# Cvičení 5.3

Vytvořte tabulku z [http://quotes.toscrape.com/](http://quotes.toscrape.com/), kde první sloupec bude jméno autora a druhý sloupec bude datum narození. autorů

In [14]:
import pandas as pd

In [15]:
url = 'http://quotes.toscrape.com'
r = requests.get(url)
soup = BeautifulSoup(r.text, 'html.parser')
author_links = soup.select('a[href*="author"]', href=True)
author_links

[<a href="/author/Albert-Einstein">(about)</a>,
 <a href="/author/J-K-Rowling">(about)</a>,
 <a href="/author/Albert-Einstein">(about)</a>,
 <a href="/author/Jane-Austen">(about)</a>,
 <a href="/author/Marilyn-Monroe">(about)</a>,
 <a href="/author/Albert-Einstein">(about)</a>,
 <a href="/author/Andre-Gide">(about)</a>,
 <a href="/author/Thomas-A-Edison">(about)</a>,
 <a href="/author/Eleanor-Roosevelt">(about)</a>,
 <a href="/author/Steve-Martin">(about)</a>]

In [16]:
authors = [['Name', 'Date of birth']]
for author in author_links:
    url = 'http://quotes.toscrape.com{}'.format(author['href'])
    r = requests.get(url)
    soup = BeautifulSoup(r.text, 'html.parser')
    authors.append([
        soup.select_one('.author-title').text, soup.select_one('.author-born-date').text
    ])
pd.DataFrame(authors[1:], columns=authors[0])    

,Name,Date of birth
0,Albert Einstein,"March 14, 1879"
1,J.K. Rowling,"July 31, 1965"
2,Albert Einstein,"March 14, 1879"
3,Jane Austen,"December 16, 1775"
4,Marilyn Monroe,"June 01, 1926"
5,Albert Einstein,"March 14, 1879"
6,André Gide,"November 22, 1869"
7,Thomas A. Edison,"February 11, 1847"
8,Eleanor Roosevelt,"October 11, 1884"
9,Steve Martin,"August 14, 1945"


# Cvičení 6.1

1. Ze stránek s mobilními telefony na heurece [https://mobilni-telefony.heureka.cz/](https://mobilni-telefony.heureka.cz/) vytvořte tabulku s následujícími sloupci:
    - Název telefona
    - Hodnocení
    - Počet recenzí
    - Minimální cena
    - Maximální cena
2. Vytvořte sloupec, kde zaokrouhlíte minimální cenu na celé tisíce
3. Seskupte telefony dle nově vytvořeného sloupce a zjistěte průměrné hodnocení pro každou skupinu

In [17]:
url = 'https://mobilni-telefony.heureka.cz/'
r = requests.get(url)
soup = BeautifulSoup(r.text, 'html.parser')

In [18]:
mobile_name = [item.text for item in soup.select('.product-container a[href*="mobilni-telefony"]')]
mobile_name

['Xiaomi Redmi Note 7 4GB/64GB',
 'Samsung Galaxy A50 A505F 4GB/128GB Dual SIM',
 'Samsung Galaxy A40 A405F Dual SIM',
 'Xiaomi Redmi Note 7 4GB/128GB',
 'Samsung Galaxy S10e G970F 128GB',
 'Apple iPhone 7 32GB',
 'Apple iPhone 8 64GB',
 'Samsung Galaxy S10 G973F 128GB',
 'Xiaomi Mi 9T 6GB/128GB',
 'Apple iPhone XS 64GB',
 'Huawei Nova 3 Dual SIM',
 'Apple iPhone XR 64GB',
 'Apple iPhone 6S 32GB',
 'Huawei P30 Lite 4GB/128GB Dual SIM',
 'Huawei Mate 20 Pro 6GB/128GB Dual SIM',
 'Samsung Galaxy S10 Plus G975F 128GB',
 'Xiaomi Redmi 6 3GB/32GB',
 'Xiaomi Redmi 7 3GB/32GB',
 'Samsung Galaxy A70 A705F Dual SIM',
 'Xiaomi Mi 9T 6GB/64GB',
 'Xiaomi Mi A3 4GB/64GB',
 'Apple iPhone X 64GB',
 'Xiaomi Redmi 6A 2GB/16GB',
 'Xiaomi Mi 9 6GB/128GB',
 'Xiaomi Redmi Note 7 3GB/32GB']

In [19]:
mobile_rating = [int(item.text.split(' (')[0].replace('%', '')) for item in soup.find_all(class_='textual')]
mobile_rating

[95,
 95,
 91,
 92,
 96,
 93,
 88,
 94,
 94,
 94,
 100,
 97,
 95,
 87,
 94,
 89,
 93,
 99,
 92,
 95,
 96,
 100,
 100,
 90,
 91,
 98,
 99]

In [20]:
mobile_review_count = [int(item.text.split(' ')[0]) for item in soup.select('.review-count a')]
mobile_review_count

[352,
 181,
 81,
 50,
 124,
 66,
 15,
 329,
 218,
 60,
 27,
 50,
 181,
 60,
 182,
 48,
 88,
 66,
 255,
 79,
 26,
 21,
 4,
 190,
 194,
 40,
 50]

In [21]:
mobile_price_min = [int(item.text.replace(' ', '')) for item in soup.find_all(class_='priceFrom')]
mobile_price_min

[4466,
 5490,
 6540,
 4990,
 4990,
 13450,
 3899,
 7666,
 12499,
 16300,
 8500,
 23900,
 5490,
 18000,
 6880,
 6490,
 11840,
 18590,
 2963,
 3290,
 8340,
 7720,
 5399,
 20890,
 1999,
 10710,
 4050]

In [22]:
mobile_price_max = [int(item.text.replace(' ', '')) for item in soup.find_all(class_='priceTo')]
mobile_price_max

[7410,
 12075,
 10868,
 7890,
 7655,
 23545,
 5796,
 16093,
 20890,
 29990,
 11343,
 36000,
 12075,
 25865,
 17990,
 9999,
 30188,
 31394,
 4150,
 4827,
 12668,
 11990,
 7184,
 29999,
 5900,
 16437,
 5913]

In [23]:
mobile = pd.DataFrame(
    zip(mobile_name, mobile_rating, mobile_review_count, mobile_price_min, mobile_price_max), 
    columns=['Mobile phone name', 'Ratings', 'Review count', 'Min price', 'Max price']
)
mobile

,Mobile phone name,Ratings,Review count,Min price,Max price
0,Xiaomi Redmi Note 7 4GB/64GB,95,352,4466,7410
1,Samsung Galaxy A50 A505F 4GB/128GB Dual SIM,95,181,5490,12075
2,Samsung Galaxy A40 A405F Dual SIM,91,81,6540,10868
3,Xiaomi Redmi Note 7 4GB/128GB,92,50,4990,7890
4,Samsung Galaxy S10e G970F 128GB,96,124,4990,7655
5,Apple iPhone 7 32GB,93,66,13450,23545
6,Apple iPhone 8 64GB,88,15,3899,5796
7,Samsung Galaxy S10 G973F 128GB,94,329,7666,16093
8,Xiaomi Mi 9T 6GB/128GB,94,218,12499,20890
9,Apple iPhone XS 64GB,94,60,16300,29990


In [24]:
mobile['Min price rounded'] = mobile['Min price'].astype(int).round(decimals=-3)
mobile

,Mobile phone name,Ratings,Review count,Min price,Max price,Min price rounded
0,Xiaomi Redmi Note 7 4GB/64GB,95,352,4466,7410,4000
1,Samsung Galaxy A50 A505F 4GB/128GB Dual SIM,95,181,5490,12075,5000
2,Samsung Galaxy A40 A405F Dual SIM,91,81,6540,10868,7000
3,Xiaomi Redmi Note 7 4GB/128GB,92,50,4990,7890,5000
4,Samsung Galaxy S10e G970F 128GB,96,124,4990,7655,5000
5,Apple iPhone 7 32GB,93,66,13450,23545,13000
6,Apple iPhone 8 64GB,88,15,3899,5796,4000
7,Samsung Galaxy S10 G973F 128GB,94,329,7666,16093,8000
8,Xiaomi Mi 9T 6GB/128GB,94,218,12499,20890,12000
9,Apple iPhone XS 64GB,94,60,16300,29990,16000


In [25]:
mobile.groupby('Min price rounded')['Ratings'].mean()

Min price rounded
2000     91.0
3000     93.5
4000     91.5
5000     95.6
6000     89.0
7000     92.5
8000     97.5
12000    93.5
13000    93.0
16000    94.0
18000    87.0
19000    99.0
21000    90.0
24000    97.0
Name: Ratings, dtype: float64

# Cvičení 6.2

Ze stránek [ministersva financí](https://www.mfcr.cz/cs/verejny-sektor/statni-rozpocet/legislativa-statniho-rozpoctu/2019/rozpocet-resortu-financi-kapitola-312-20-33908) vyextrahujte tabulku rozpočtu.

In [26]:
url = 'https://www.mfcr.cz/cs/verejny-sektor/statni-rozpocet/legislativa-statniho-rozpoctu/2019/rozpocet-resortu-financi-kapitola-312-20-33908'
r = requests.get(url)
soup = BeautifulSoup(r.text, 'html.parser')

In [27]:
budget = pd.read_html(str(soup.find('table')), header=[0, 1])[0]
budget

,Unnamed: 0_level_0,Unnamed: 1_level_0,Unnamed: 2_level_0,2019,2020,2021
,Souhrnné ukazatele,Souhrnné ukazatele,Souhrnné ukazatele,Souhrnné ukazatele,Souhrnné ukazatele,Souhrnné ukazatele
0,Příjmy celkem,Příjmy celkem,Příjmy celkem,5 254 899 121,5 151 745 436,5 147 764 436
1,Výdaje celkem,Výdaje celkem,Výdaje celkem,23 617 964 238,23 182 199 324,22 959 559 324
2,Specifické ukazatele - příjmy,Specifické ukazatele - příjmy,Specifické ukazatele - příjmy,Specifické ukazatele - příjmy,Specifické ukazatele - příjmy,Specifické ukazatele - příjmy
3,Daňové příjmy1),Daňové příjmy1),Daňové příjmy1),32 900 000,32 900 000,29 900 000
4,Příjmy z pojistného na sociální zabezpečení a ...,Příjmy z pojistného na sociální zabezpečení a ...,Příjmy z pojistného na sociální zabezpečení a ...,763 755 436,763 755 436,763 755 436
5,v tom:,pojistné na důchodové pojištění,pojistné na důchodové pojištění,678 893 721,678 893 721,678 893 721
6,NaN,pojistné na nemocenské pojištění a příspěvek n...,pojistné na nemocenské pojištění a příspěvek n...,84 861 715,84 861 715,84 861 715
7,"Nedaňové příjmy, kapitálové příjmy a přijaté t...","Nedaňové příjmy, kapitálové příjmy a přijaté t...","Nedaňové příjmy, kapitálové příjmy a přijaté t...",4 458 243 685,4 355 090 000,4 354 109 000
8,v tom:,příjmy z rozpočtu Evropské unie bez společné z...,příjmy z rozpočtu Evropské unie bez společné z...,272 700 759,0,0
